### Q1.Design a Sudoku puzzle where the board consists of 81 squares, some of which are initially filled with digits from 1 to 9.  

The puzzle is to fill in all the remaining squares such that no digit appears twice in any row, column, or 3 × 3 box.  
A row, column, or box is called a unit.  

1.	Represent the Sudoku problem in  CSP by identifying the variable, domain, and constraint.
2.	Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.
3.	Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?
4.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?


refrence : 
1. https://stackoverflow.com/questions/45471152/how-to-create-a-sudoku-puzzle-in-python 

In [26]:
import numpy as np
from pprint  import pprint 
import sys,warnings
warnings.filterwarnings("ignore", category=UserWarning)  

In [ ]:

base  = 3
side  = base*base

# pattern for a baseline valid solution
def pattern(r,c): return (base*(r%base)+r//base+c)%side

# randomize rows, columns and numbers (of valid base pattern)
from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

# produce board using randomized baseline pattern
solution = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
# print(solution)
board = np.array(solution)

# original_array = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# # Deep copy of the array
# copied_array = original_array.copy()

squares = side*side
empties = squares * 3//4
for p in sample(range(squares),empties):
    board[p//side][p%side] = 0

numSize = len(str(side))
# for line in board:
#     print(*(f"{n or '.':{numSize}} " for n in line))
# print(solution)

In [15]:
def shortSudokuSolve(board):
    side   = len(board)
    base   = int(side**0.5)
    board  = [n for row in board for n in row ]
    blanks = [i for i,n in enumerate(board) if n==0 ]
    cover  = { (n,p):{*zip([2*side+r, side+c, r//base*base+c//base],[n]*(n and 3))}
                for p in range(side*side) for r,c in [divmod(p,side)] for n in range(side+1) }
    used   = set().union(*(cover[n,p] for p,n in enumerate(board) if n))
    placed = 0
    while placed>=0 and placed<len(blanks):
        pos        = blanks[placed]
        used      -= cover[board[pos],pos]
        board[pos] = next((n for n in range(board[pos]+1,side+1) if not cover[n,pos]&used),0)
        used      |= cover[board[pos],pos]
        placed    += 1 if board[pos] else -1
        if placed == len(blanks):
            solution = [board[r:r+side] for r in range(0,side*side,side)]
            yield solution
            placed -= 1

In [16]:
from IPython.display import display, HTML

def sudoku_to_html(board):
    """
    Generates an HTML table representation of a Sudoku board with thick edges for subgrids.

    Args:
        board (list of list of int): 2D list representing the Sudoku board.
                                     Empty cells should be represented by 0 or '.'.

    Returns:
        str: HTML string for the Sudoku board.
    """
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .thick-border-top { border-top: 3px solid black; }
        .thick-border-left { border-left: 3px solid black; }
    </style>
    <table>
    """

    for i, row in enumerate(board):
        html += "<tr>"
        for j, cell in enumerate(row):
            # Apply thicker borders for subgrid boundaries
            classes = []
            if i % 3 == 0 and i != 0:
                classes.append("thick-border-top")
            if j % 3 == 0 and j != 0:
                classes.append("thick-border-left")
            
            class_attr = f'class="{" ".join(classes)}"' if classes else ""
            html += f"<td {class_attr}>{cell if cell != 0 else ''}</td>"
        html += "</tr>"

    html += "</table>"
    return html

In [17]:
# Generate and display the Sudoku board as HTML
html_output = sudoku_to_html(board)
display(HTML(html_output))

html_output = sudoku_to_html(solution)
display(HTML(html_output))


,8,,9,2,,5,6,7
,,,8,,,,,
,,,5,,,,,
,,,2,,,,,
3,,5,1,,,,,
7,,9,6,,,,4,8
,4,,7,,,,,
,,,,,,,5,
,,,,8,,,,


1,8,3,9,2,4,5,6,7
6,5,7,8,1,3,9,2,4
2,9,4,5,6,7,8,1,3
4,1,8,2,7,9,6,3,5
3,6,5,1,4,8,2,7,9
7,2,9,6,3,5,1,4,8
9,4,1,7,5,2,3,8,6
8,3,6,4,9,1,7,5,2
5,7,2,3,8,6,4,9,1


In [18]:
board

array([[0, 8, 0, 9, 2, 0, 5, 6, 7],
       [0, 0, 0, 8, 0, 0, 0, 0, 0],
       [0, 0, 0, 5, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0],
       [3, 0, 5, 1, 0, 0, 0, 0, 0],
       [7, 0, 9, 6, 0, 0, 0, 4, 8],
       [0, 4, 0, 7, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 8, 0, 0, 0, 0]])

In [22]:
def row_constraint(assignment, row):
    row_values = [assignment.get((row, c)) for c in range(9) if assignment.get((row, c)) is not None]
    return len(row_values) == len(set(row_values))

def col_constraint(assignment, col):
    col_values = [assignment.get((r, col)) for r in range(9) if assignment.get((r, col)) is not None]
    return len(col_values) == len(set(col_values))

def block_constraint(assignment, block_row, block_col):
    block_values = []
    for r in range(block_row * 3, block_row * 3 + 3):
        for c in range(block_col * 3, block_col * 3 + 3):
            value = assignment.get((r, c))
            if value is not None:
                block_values.append(value)
    return len(block_values) == len(set(block_values))

1. Represent the Sudoku problem in CSP by identifying the variable, domain, and constraint.

In [39]:
def represent_sudoku_csp(board):
    """
    Represents a Sudoku board as a Constraint Satisfaction Problem (CSP) with detailed variable and constraint structures.

    Args:
        board (list of list of int): A 9x9 2D list representing the Sudoku board.
                                     Empty cells are represented by 0.

    Returns:
        dict: A dictionary where each key is a tuple (row, col) representing a cell, and the value is another dictionary containing:
            - 'row' (int): The row index of the cell.
            - 'col' (int): The column index of the cell.
            - 'value' (int or None): The value assigned to the cell, or None if the cell is empty.
            - 'domain' (list of int): The possible values that can be assigned to the cell.
                                      For pre-filled cells, this is a single-value list.
            - 'constraints' (dict): A dictionary of constraint functions for the cell, including:
                - 'row': A function to enforce the row constraint.
                - 'col': A function to enforce the column constraint.
                - 'box': A function to enforce the 3x3 subgrid constraint.

    Notes:
        - This function provides a more detailed representation of the CSP, where each variable (cell) is associated with its constraints and domain.
        - The constraints ensure that no number is repeated in the same row, column, or 3x3 subgrid.
        - This representation can be used with CSP solvers to solve the Sudoku puzzle.
    """
    side = len(board)
    variables = {}

    for row in range(side):
        for col in range(side):
            variable = {
                'row': row,
                'col': col,
                'value': board[row][col], # initial value, zero represents not filled yet
                'domain': [board[row][col]] if board[row][col]>0 else list(range(1, 10)), # initial domains
                'PreFilled': True if board[row][col]>0 else False, # value given by the puzzle
                'constraints': {
                    'row': row_constraint, # function that returns true if the value does not exist in the row
                    'col': col_constraint, # function that returns true if the value does not exist in the column
                    'box': block_constraint # function that returns true if the value does not exist in the block
                }
            }
        
            variables[(row, col)] = variable
    return variables

In [40]:
variables_dict = represent_sudoku_csp(board)
pprint(variables_dict[list(variables_dict.keys())[0]])
# CSP representation of the first cell of the Sudoku board

{'PreFilled': False,
 'col': 0,
 'constraints': {'box': <function block_constraint at 0x7f7a37c53ec0>,
                 'col': <function col_constraint at 0x7f7a37c527a0>,
                 'row': <function row_constraint at 0x7f7a37c52520>},
 'domain': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'row': 0,
 'value': 0}


2. Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.